# **Assignment 4 ROB313 - Neural Networks**

In [2]:
# Import relevant libraries
import autograd.numpy as np
from autograd.scipy.special import logsumexp
import autograd.numpy.random as npr
from autograd import grad
from autograd.misc.optimizers import adam
import os
import sys
import matplotlib.pyplot as plt

# path to data
sys.path.append(os.path.join(os.getcwd(), "../../../data"))
from data.data_utils import load_dataset

In [ ]:
# Neuron Initialization and Activations

def init_randn(m, n, rs=npr.RandomState(0)):
    """ init mxn matrix using small random normal"""
    return 0.1 * rs.randn(m, n)

def init_xavier(m, n, rs=npr.RandomState(0)):
    """Init mxn matrix using Xavier intialization - initialization of weights"""
    return rs.randn(m,n)/np.sqrt(n)

def init_net_params(layer_sizes, init_fcn, rs=npr.RandomState(0)):
    """ inits a (weights, biases) tuples for all layers using the intialize function (init_fcn)"""
    return [
        (init_fcn(m, n), np.zeros(n))  # weight matrix  # bias vector
        for m, n in zip(layer_sizes[:-1], layer_sizes[1:])
    ]

def relu(x):
    return np.maximum(0, x)

In [ ]:
# Neural network training and prediction

def neural_net_predict(params, inputs):
    """Implements a deep neural network for classification.
       params is a list of (weights, bias) tuples.
       inputs is an (N x D) matrix.
       returns normalized class log-probabilities."""
    for W, b in params:
        outputs = np.dot(inputs, W) + b
        inputs = relu(outputs)
    return outputs - logsumexp(outputs, axis=1, keepdims=True)

def mean_log_like(params, inputs, targets):
    """ TODO: return the log-likelihood / the number of inputs 
    """
    raise NotImplementedError("mean_log_like function not completed (see Q3).")

def accuracy(params, inputs, targets):
    """ return the accuracy of the neural network defined by params
    """
    target_class = np.argmax(targets, axis=1)
    predicted_class = np.argmax(neural_net_predict(params, inputs), axis=1)
    return np.mean(predicted_class == target_class)

In [ ]:
# Example of Code Usage

# loading data
x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset("mnist_small")

# initializing parameters
layer_sizes = [784, 200, 10]
params = init_net_params(layer_sizes, init_randn)

# setting up training parameters
num_epochs = 5
learning_rate = 1e-1
batch_size = 256

# Constants for batching
num_batches = int(np.ceil(len(x_train) / batch_size))
rind = np.arange(len(x_train))
npr.shuffle(rind)

def batch_indices(iter):
    idx = iter % num_batches
    return rind[slice(idx * batch_size, (idx + 1) * batch_size)]

# Define training objective
def objective(params, iter):
    # get indices of data in batch
    idx = batch_indices(iter)
    return -mean_log_like(params, x_train[idx], y_train[idx])

# Get gradient of objective using autograd.
objective_grad = grad(objective)

print("     Epoch     |    Train accuracy  |       Test accuracy  ")

# Dictionary to store train/val history
opt_history = {
    "train_nll": [],
    "val_nll": [],
}

def callback(params, iter, gradient):
    if iter % num_batches == 0:
        # record training & val. accuracy every epoch
        opt_history["train_nll"].append(-mean_log_like(params, x_train, y_train))
        opt_history["val_nll"].append(-mean_log_like(params, x_valid, y_valid))
        train_acc = accuracy(params, x_train, y_train)
        val_acc = accuracy(params, x_valid, y_valid)
        print("{:15}|{:20}|{:20}".format(iter // num_batches, train_acc, val_acc))

# We will optimize using Adam (a variant of SGD that makes better use of
# gradient information).
opt_params = adam(
    objective_grad,
    params,
    step_size=learning_rate,
    num_iters=num_epochs * num_batches,
    callback=callback,
)

# Plotting the train and validation negative log-likelihood
plt.plot(opt_history["train_nll"])
plt.plot(opt_history["val_nll"])
plt.show()

**Question 1**
- Epsilon, the variance of the linear activation function must be equal to 1/sqrt(D^2)
- In this case, D is the number of inputs to a neuron

**Question 2**
- The logsumexp function from scipy computes the log of the sum of exponentials of input elements for us. We have used the logsumexp function instead of a more naive approach (manual computation) to get the predicted log probabilities for each class because it does this computation in a more stable way. This function can help us prevent underflow/overflow errors, which is one issue that we experienced within assignment 3 when working with log likelihoods. For instance, the naive approach could crash in multiple instances due to division by 0 or when dealing with inf/nan values, thus causing numerical instability. Overall, the logsumexp function prevents this issue from occuring, therefore making our program more robust.